In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
sz = 128

In [3]:
# Step 1 - Building the CNN

# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
#classifier.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
#classifier.add(MaxPooling2D(pool_size=(2, 2)))


In [4]:
# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2


In [5]:
# Compiling the CNN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Step 2 - Preparing the train/test data and training the model
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3686528   
_________________________________________________________________
dropout (Dropout)            (None, 128)               0

In [7]:
from keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('data2/train',
                                                 target_size=(sz, sz),
                                                 batch_size=10,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data2/test',
                                            target_size=(sz , sz),
                                            batch_size=10,
                                            color_mode='grayscale',
                                            class_mode='categorical') 
print(len(training_set))
print(len(test_set))

Found 12845 images belonging to 27 classes.
Found 4268 images belonging to 27 classes.
1285
427


In [9]:
train_imgs=len(training_set)
test_imgs=len(test_set)
classifier.fit_generator(
        training_set,
        steps_per_epoch=train_imgs, # No of images in training set
        epochs=5,
        validation_data=test_set,
        validation_steps=test_imgs)  #test set

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
1285/1285 [==============================] - 577s 449ms/step - loss: 2.0231 - accuracy: 0.3823 - val_loss: 0.3237 - val_accuracy: 0.8906
Epoch 2/5
1285/1285 [==============================] - 198s 154ms/step - loss: 0.6801 - accuracy: 0.7678 - val_loss: 0.1029 - val_accuracy: 0.9698
Epoch 3/5
1285/1285 [==============================] - 210s 164ms/step - loss: 0.4491 - accuracy: 0.8495 - val_loss: 0.0646 - val_accuracy: 0.9784
Epoch 4/5
1285/1285 [==============================] - 212s 165ms/step - loss: 0.3371 - accuracy: 0.8852 - val_loss: 0.0311 - val_accuracy: 0.9937
Epoch 5/5
1285/1285 [==============================] - 210s 163ms/step - loss: 0.2847 - accuracy: 0.9102 - val_loss: 0.0240 - val_accuracy: 0.9923


In [10]:
# Saving the model
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-bw.h5')
print('Weights saved')

Model Saved
Weights saved
